In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int8",
        "month": "int8", "year": "int16", "snap_CA": "int8", 'snap_TX': 'int8', 'snap_WI': 'int8' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
pd.options.display.max_columns = 50

In [4]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [5]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("../input/csv/sell_prices.csv.gz", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("../input/csv/calendar.csv.gz", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("../input/csv/sales_train_validation.csv.gz", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [6]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [7]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [8]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

Wall time: 36.2 s


(40718219, 22)

In [9]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.77


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
id              object
item_id         int16
dept_id         int16
store_id        int16
cat_id          int16
state_id        int16
d               object
sales           float32
date            datetime64[ns]
wm_yr_wk        int16
weekday         int16
wday            int8
month           int8
year            int16
event_name_1    int16
event_type_1    int16
event_name_2    int16
event_type_2    int16
snap_CA         int8
snap_TX         int8
snap_WI         int8
sell_price      float32
dtypes: datetime64[ns](1), float32(2), int16(12), int8(5), object(2)
memory usage: 2.6+ GB


In [11]:
%%time

create_fea(df)
df.shape

Wall time: 2min 37s


(40718219, 31)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 31 columns):
id              object
item_id         int16
dept_id         int16
store_id        int16
cat_id          int16
state_id        int16
d               object
sales           float32
date            datetime64[ns]
wm_yr_wk        int16
weekday         int16
wday            int16
month           int16
year            int16
event_name_1    int16
event_type_1    int16
event_name_2    int16
event_type_2    int16
snap_CA         int8
snap_TX         int8
snap_WI         int8
sell_price      float32
lag_7           float32
lag_28          float32
rmean_7_7       float32
rmean_28_7      float32
rmean_7_28      float32
rmean_28_28     float32
week            int16
quarter         int16
mday            int16
dtypes: datetime64[ns](1), float32(8), int16(17), int8(3), object(2)
memory usage: 3.8+ GB


In [13]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.97,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.34,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.48,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.50,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.77,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13


In [14]:
df.dropna(inplace = True)
df.shape

(39041269, 31)

In [15]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [17]:
df = reduce_mem_usage(df)

Mem. usage decreased to 2643.52 Mb (29.7% reduction)


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39041269 entries, 869062 to 40718218
Data columns (total 31 columns):
id              object
item_id         int16
dept_id         int8
store_id        int8
cat_id          int8
state_id        int8
d               object
sales           float16
date            datetime64[ns]
wm_yr_wk        int16
weekday         int8
wday            int8
month           int8
year            int16
event_name_1    int8
event_type_1    int8
event_name_2    int8
event_type_2    int8
snap_CA         int8
snap_TX         int8
snap_WI         int8
sell_price      float16
lag_7           float16
lag_28          float16
rmean_7_7       float16
rmean_28_7      float16
rmean_7_28      float16
rmean_28_28     float16
week            int8
quarter         int8
mday            int8
dtypes: datetime64[ns](1), float16(8), int16(3), int8(17), object(2)
memory usage: 2.6+ GB


In [20]:
gc.collect()

1344

In [21]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [22]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [23]:
%%time

np.random.seed(1234)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

Wall time: 23.2 s


In [24]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

90

In [25]:
params = {
        "objective" : "poisson",
        "metric" :"mse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["mse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [26]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

C:\Users\yohei\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's l2: 8.56134
[40]	valid_0's l2: 6.617
[60]	valid_0's l2: 6.16325
[80]	valid_0's l2: 6.04724
[100]	valid_0's l2: 5.99348
[120]	valid_0's l2: 5.94745
[140]	valid_0's l2: 5.91191
[160]	valid_0's l2: 5.87038
[180]	valid_0's l2: 5.83098
[200]	valid_0's l2: 5.79929
[220]	valid_0's l2: 5.76671
[240]	valid_0's l2: 5.7472
[260]	valid_0's l2: 5.72227
[280]	valid_0's l2: 5.697
[300]	valid_0's l2: 5.67969
[320]	valid_0's l2: 5.66108
[340]	valid_0's l2: 5.63469
[360]	valid_0's l2: 5.61627
[380]	valid_0's l2: 5.59389
[400]	valid_0's l2: 5.58037
[420]	valid_0's l2: 5.56906
[440]	valid_0's l2: 5.55113
[460]	valid_0's l2: 5.53911
[480]	valid_0's l2: 5.52823
[500]	valid_0's l2: 5.52149
[520]	valid_0's l2: 5.50584
[540]	valid_0's l2: 5.49401
[560]	valid_0's l2: 5.48179
[580]	valid_0's l2: 5.46859
[600]	valid_0's l2: 5.45375
[620]	valid_0's l2: 5.43262
[640]	valid_0's l2: 5.42174
[660]	valid_0's l2: 5.4121
[680]	valid_0's l2: 5.40583
[700]	valid_0's l2: 5.39963
[720]	valid_0's l2: 5.3894
[74

In [27]:
m_lgb.save_model("model.lgb")

In [28]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [29]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.890406,0.849721,0.837108,0.827008,1.060738,1.296211,1.201395,0.969319,0.956976,0.959099,0.961707,1.045235,1.309049,1.165278,0.973172,0.915224,0.953120,0.970818,1.085930,1.363375,1.259133,0.958556,0.863561,0.845287,0.867933,1.025206,1.282610,1.262687
1,FOODS_1_001_CA_2_validation,0.923992,0.931533,0.913005,1.012566,1.106058,1.176863,1.487573,0.927058,0.924876,0.899441,0.946906,1.099094,1.364347,1.180002,0.999799,0.920639,0.970037,0.976472,1.175197,1.533026,1.414818,1.008661,0.972896,0.963762,0.982368,1.147125,1.511884,1.306500
2,FOODS_1_001_CA_3_validation,1.137266,1.066436,0.927218,0.892963,0.958631,1.166119,1.271915,1.071183,1.087580,0.988619,0.996495,1.058144,1.395315,1.223349,1.089934,1.088092,1.000331,1.023336,1.106117,1.531362,1.565947,1.124175,1.075494,0.923863,0.918545,1.002742,1.291091,1.316364
3,FOODS_1_001_CA_4_validation,0.383028,0.341178,0.359209,0.368474,0.411835,0.463580,0.509493,0.392960,0.412308,0.400310,0.402096,0.390109,0.418923,0.368550,0.355932,0.357898,0.388353,0.398151,0.445627,0.466121,0.476165,0.366694,0.349811,0.353258,0.362669,0.423529,0.450631,0.459133
4,FOODS_1_001_TX_1_validation,0.197477,0.189502,0.184222,0.188166,0.175379,0.192955,0.203963,0.472994,0.456582,0.435934,0.448782,0.462386,0.457038,0.389728,0.386173,0.423417,0.345351,0.347441,0.377136,0.359594,0.342286,0.288533,0.278547,0.274914,0.277297,0.292615,0.337801,0.327449
5,FOODS_1_001_TX_2_validation,0.463179,0.428609,0.436889,0.394061,0.449246,0.519349,0.539382,0.441443,0.446697,0.415879,0.500766,0.489618,0.574127,0.503272,0.451955,0.426067,0.466856,0.447933,0.518372,0.563400,0.572167,0.424602,0.415775,0.423207,0.416745,0.466855,0.530524,0.523127
6,FOODS_1_001_TX_3_validation,0.392894,0.354000,0.363986,0.410830,0.461857,0.473142,0.544551,0.450736,0.505412,0.462041,0.494791,0.518997,0.540467,0.448723,0.444201,0.416254,0.450253,0.488296,0.507842,0.553231,0.536244,0.438779,0.413166,0.428251,0.432562,0.474235,0.514246,0.496355
7,FOODS_1_001_WI_1_validation,0.327424,0.353792,0.341278,0.347216,0.435506,0.710320,0.667473,0.490919,0.460565,0.433861,0.536807,0.624040,0.751774,0.565168,0.518087,0.506991,0.549495,0.552784,0.677729,0.826453,0.773399,0.546516,0.541807,0.553636,0.564264,0.679312,0.942645,0.812037
8,FOODS_1_001_WI_2_validation,0.303620,0.318991,0.334569,0.333619,0.411441,0.461403,0.429671,0.429304,0.495196,0.415242,0.458737,0.498745,0.508295,0.399428,0.437161,0.390170,0.477544,0.462646,0.464010,0.523791,0.502792,0.372133,0.378598,0.373439,0.381052,0.399132,0.456475,0.415447
9,FOODS_1_001_WI_3_validation,0.235891,0.229447,0.213403,0.222834,0.279188,0.348588,0.354558,0.352406,0.375903,0.328267,0.384118,0.435813,0.426158,0.366609,0.346185,0.323405,0.358047,0.343006,0.389424,0.449169,0.435327,0.313239,0.298852,0.306946,0.307534,0.358069,0.422380,0.364944


In [30]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [31]:
sub.shape

(60980, 29)